# Desarrollo e implementación de un sistema de busqueda, casos y/o similitud en una base de datos orientada a grafos

Se desea generar métodos inteligentes para encontrar información basada en un ejemplo aplicando datos del presidente obtenidos de redes sociales, etc..

Pasos a seguir:
1. Realizar una extracción de datos  en base a palabras claves del presidente.
2. Migrar esta información a la base de grafos Neo4j, en virtud de ello, se deberá tener al menos 2000 nodos.
3. Implementar los algoritmos inteligentes dentro de la base de grafos.
4. Consultar información y/o resultados del grafo aplicando los algoritmos inteligentes.
6. Agregar conclusiones y recomendaciones.

In [51]:
from neo4j import GraphDatabase

class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()      
    
    def crear_Presidente(self, tx, nombre):
        tx.run("CREATE (:Presidente {nombre: $nombre})", nombre=nombre)  
        
    def crear_Tweets(self, tx, nombre):
        tx.run("CREATE (:Tweet {nombre: $nombre})", nombre=nombre)   
        
    def crear_Tweet(self, tx, nombre, fuente, retuits):
        tx.run("CREATE (:Noticias {nombre: $nombre, fuente: $fuente, retuits: $retuits})", nombre=nombre,fuente=fuente,retuits=retuits)        
        
    
#realciones central - alcandes
    
    def crear_relacion_TweetAl(self, tx, nombre_alcalde, nombre_tweet):
        tx.run("MATCH (a:Presidente {nombre: $nombre_alcalde}) "
               "MATCH (b:Tweet {nombre: $nombre_tweet}) "
               "MERGE (a)-[:tweet_Alcalde]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_tweet=nombre_tweet)
    
#realciones alcalde - noticia

    def crear_relacion_TitNot(self, tx, nombre_titulo,nombre_noticia,l):
        tx.run("MATCH (a:Tweet {nombre: $nombre_titulo}) "
               "MATCH (b:Noticias {nombre: $nombre_noticia}) "
               "MERGE (a)-[:Tweet_Noticia {retuits:[$l]}]->(b)",
               nombre_titulo=nombre_titulo, nombre_noticia=nombre_noticia,l=l)

print("Ejecucion correcta")

Ejecucion correcta


In [53]:
import tweepy
import time
import random

neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'integrador2')
with neo4j._driver.session() as session:

    session.write_transaction(neo4j.crear_Presidente , "Guillermo Lasso")  
    
# 4 cadenas para la autenticacion
    consumer_key = "rwNrplwdiEpTFAc3WVeJmRHNI"
    consumer_secret = "vH6Hfo5kZHJHycymBGKQuFoqWWaKm9bXDb4jmvnCAVnEzW9h3d"
    access_token = "1302969322533519360-z5rquE3xp6bEaGROARjADTeAoqrrEt"
    access_token_secret = "yCm4m9MolHrS0QHX59N8pO11KFOMtbkiztb1acIttIw8V"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    # con este objeto realizaremos todas las llamadas al API
    api = tweepy.API(auth,
                     wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True)
    a=0
    h=1
    nodo="TWEET"
    l=0
    
    session.write_transaction(neo4j.crear_Tweets , nodo)
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(440):
        l=random.randrange(1, 20, 1)
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
           
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1 
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
                
    time.sleep(65) 
    print("fin uno")

    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(440):
        l=random.randrange(1, 20, 1)
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
           
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1 
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
                
    time.sleep(65) 
    print("fin dos uno")
    
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(440):
        l=random.randrange(1, 20, 1)
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
           
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1 
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
                
    time.sleep(65) 
    print("fin dos")
    
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(440):
        l=random.randrange(1, 20, 1)
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
           
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1 
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
                
    time.sleep(65) 
    print("fin tres")
    
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(440):
        l=random.randrange(1, 20, 1)
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
           
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1 
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"],tweet._json["retweet_count"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
                
    time.sleep(65) 
    print("fin cuatro")
    print("fin final")
      

fin uno
fin dos uno
fin dos
fin tres
fin cuatro
fin final


In [ ]:
 time.sleep(65) 
    print("fin uno")
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(430):
        
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
    
    time.sleep(65) 
    print("fin dos")
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(430):
        
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
    
    time.sleep(65) 
    print("fin tres")
    for tweet in tweepy.Cursor(api.search, q="Guillermo Lasso", tweet_mode="extended").items(430):
        
        #print (json.dumps(tweet._json, indent=4))
        if(a==1):
            nodo="TWEET"+str(h)
            session.write_transaction(neo4j.crear_Tweets , nodo)
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
            a=0
            h=h+1
        else:
            a=a+1
            session.write_transaction(neo4j.crear_Tweet , tweet._json["full_text"], tweet._json["user"]["screen_name"], tweet._json["retweet_count"])
            session.write_transaction(neo4j.crear_relacion_TitNot, nodo, tweet._json["full_text"]) 
            session.write_transaction(neo4j.crear_relacion_TweetAl, "Guillermo Lasso", nodo)
    
    time.sleep(65) 
    print("fin cuatro")